In [104]:
from client import Client
from global_model import Global_model
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import numpy as np

import random

In [105]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [106]:
selected_model = Supported_modles.logistic_regression
RANDOM_NUMBER_OF_CLIENTS = 3

In [107]:
#NODEs
client1 = Client("node1","0.0.0.0")
dataset = client1.load_data("../../datasets/UNSW_NB15_training-set.csv")
client2 = Client("node2","0.0.0.0")
client3 = Client("node3","0.0.0.0")
client4 = Client("node4","0.0.0.0")
client5 = Client("node5","0.0.0.0")

dataset = shuffle(dataset)

clients = [client1, client2, client3, client4, client5]

client1.preprocess_data(dataset)

X = client1.x
y = client1.y

client1.x = X[:20000]
client1.y = y[:20000]

client2.x = X[20000:40000]
client2.y = y[20000:40000]

client3.x = X[40000:50000]
client3.y = y[40000:50000]

client4.x = X[50000:60000]
client4.y = y[50000:60000]

client5.x = X[60000:]
client5.y = y[60000:]

In [108]:
f1_before = []
for client in clients:
    
    client.split_data()
    client.train_model(selected_model)
    print(f'Model accuracy: {client.accuracy}')
    print(f'Model f1 score: {client.F1}')
    f1_before.append(client.F1)
    print(f'Total size: {client.x.shape[0]}')
    print(f'Totla sum of malicious packets: {client.y.sum()}')

Model accuracy: 0.8562121212121212
Model f1 score: 0.8710772992799891
Total size: 13400
Totla sum of malicious packets: 7400
Model accuracy: 0.8501515151515151
Model f1 score: 0.8628864550117842
Total size: 13400
Totla sum of malicious packets: 7387
Model accuracy: 0.8545454545454545
Model f1 score: 0.8671831765356945
Total size: 6700
Totla sum of malicious packets: 3658
Model accuracy: 0.8636363636363636
Model f1 score: 0.878771551724138
Total size: 6700
Totla sum of malicious packets: 3751
Model accuracy: 0.8618724559023067
Model f1 score: 0.8733830845771144
Total size: 14962
Totla sum of malicious packets: 8269


In [109]:
global_model = Global_model(selected_model)
global_model.init_global_model(client1.x.shape[1], selected_model)

In [114]:
number_of_rounds = 5
batch_size = 20

for _ in range(number_of_rounds):

    print(f'Starting new round!')
    f1s = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)
    data_weights = arr = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)
    total_data = 0

    applicable_clients = random.sample((clients), RANDOM_NUMBER_OF_CLIENTS)

    for client in applicable_clients:
        X_train, X_test, y_train, y_test = train_test_split(client.x, client.y, shuffle=True, train_size=batch_size, stratify=y, random_state=seed+i+r)
        X_train = prep.fit_transform(X_train)

    for client in applicable_clients:
        print(f'Client name: {client.name}')
        client.test_model_f1()
        f1s = np.append(f1s, client.F1)
        #sum data
        data_weights = np.append(data_weights, client.x.shape[0])
    
    # print(f'Weights: {f1s/sum(f1s)}')
    # print(f'weigths data: {data_weights/sum(data_weights)}')
    
    round_weights = (f1s/sum(f1s) + data_weights/sum(data_weights))/2
    
    print(round_weights)

    global_model.update_global_model(applicable_clients,round_weights,selected_model)
    
    for client in applicable_clients:
        f1_local = client.test_model_f1()
        if selected_model == Supported_modles.MLP_classifier:
            int_local = client.model.intercepts_
            coef_local = client.model.coefs_
            # if client in applicable_clients:
            #     print(f'Intercepts local: {int_local}')
            #     print(f'Intercepts fede: {global_model.model.intercepts_}')
            client.model.intercepts_ = global_model.model.intercepts_
            client.model.coefs_ = global_model.model.coefs_
        else:
            int_local = client.model.intercept_
            coef_local = client.model.coef_
            client.model.intercept_ = global_model.model.intercept_
            client.model.coef_ = global_model.model.coef_
        print("f1 local: ", f1_local)

        f1_fede = client.test_model_f1()
        print("f1 fede: ", f1_fede)
        if f1_fede < f1_local:
            # local was better set to local values
            if selected_model == Supported_modles.MLP_classifier:
                client.model.intercepts_ = int_local
                client.model.coefs_ = coef_local
            else:
                client.model.intercept_ = int_local
                client.model.coef_ = coef_local
            print("f1 local was better")
        elif f1_fede == f1_local:
            print("the same")
        else:
            print("f1 fede was better")
            client.F1 = f1_fede
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Starting new round!
Client name: node1
Client name: node2
Client name: node3
[0.36675173 0.36704389 0.26620438]
f1 local:  0.8749489726493401
f1 fede:  0.8749489726493401
the same
f1 local:  0.8764819409980701
f1 fede:  0.8763611302549966
f1 local was better
f1 local:  0.8720770288858322
f1 fede:  0.872317006053935
f1 fede was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node3
Client name: node1
Client name: node5
[0.26197651 0.35802371 0.37999978]
f1 local:  0.872317006053935
f1 fede:  0.8720066061106523
f1 local was better
f1 local:  0.8749489726493401
f1 fede:  0.8745578231292516
f1 local was better
f1 local:  0.8733830845771144
f1 fede:  0.8731027618810651
f1 local was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node4
Client name: node5
Client name: node2
[0.26334809 0.37916922 0.35748269]
f1 local:  0.8839114995902759
f1 fede:  0.8836

In [115]:
i = 0
for client in clients:
    f1_before[i]
    print(f'Difference {client.F1-f1_before[i]}')
    i = i + 1

Difference 0.003871673369351014
Difference 0.013595485986285905
Difference 0.005684187250766803
Difference 0.005139947866137917
Difference 0.00010864324972970074
